# ------ **Import Subroutines and Settings** -------

# Set up for importing What in the **** data ie: **df_mf_ib77_nn** to add to the main data frame **df_mf_ib_tst** trying 97

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [4]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")

NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# Pre-concatinate processing import and prepare 
1. **"df_m_ib_tst"**
2. **"df_m_ib77_nn"**


## def functions called in data importing & refinment.

In [5]:
media_lst = [
    "timestamp",
    "dtv",
    "ib_id",
    "cls",
    "cmmnts"
]

In [6]:
# 3rd version def drop_duplicates_by_test_time(df, keep='first', log=True):
def drop_duplicates_by_test_time(df, keep='first', log=True):
    """
    Removes duplicate rows based on the 'Test Date / Time' column.
    Keeps only the first (or last) occurrence.
    """
    df = df.copy()

    # Identify duplicate timestamps (beyond the one we keep)
    dupes = (
        df.loc[df.duplicated(subset=['Test Date / Time'], keep=keep), 'Test Date / Time']
        .astype(str)
        .values
        .tolist()
    )

    # Drop duplicate rows
    df = df.drop_duplicates(subset=['Test Date / Time'], keep=keep)

    # Optional logging
    if log and dupes:
        print("Removed duplicate rows for timestamps:", dupes)

    return df


In [7]:
# def strip_numbers_from_columns(df):
import re

def strip_numbers_from_columns(df):
    """
    Removes leading/trailing numbers and any leftover separators
    so that cases like '1.0ID' become 'ID'.
    """
    df = df.copy()
    new_cols = {}

    for col in df.columns:
        cleaned = col

        # Remove leading numbers + separators
        cleaned = re.sub(r'^\d+[\s\-\_\.:]*', '', cleaned)

        # Remove trailing numbers + separators
        cleaned = re.sub(r'[\s\-\_\.:]*\d+$', '', cleaned)

        # Remove leftover leading/trailing punctuation (.,-_:) after number removal
        cleaned = re.sub(r'^[\.\-\_\:]+', '', cleaned)
        cleaned = re.sub(r'[\.\-\_\:]+$', '', cleaned)

        new_cols[col] = cleaned

    return df.rename(columns=new_cols)


In [8]:
# Use drop duplicate function If duplicates are found
def drop_duplicate_columns(df, keep='first', log=True):
    """
    Removes duplicate column names from a DataFrame, keeping only the first
    (or last) occurrence. Useful after column-cleaning steps that may cause
    collisions. good i'm moving this around because I want to go ahead and do the I'm talking too

    Parameters
    ----------
    df : pandas.DataFrame
        Input DataFrame.
    keep : {'first', 'last'}, default 'first'
        Which duplicate to keep.
    log : bool, default True
        Whether to print which columns were removed.

    Returns
    -------
    pandas.DataFrame
        DataFrame with duplicate columns removed.
    """
    df = df.copy()

    # Identify duplicates beyond the one we keep
    dupes = df.columns[df.columns.duplicated(keep=keep)].tolist()

    # Drop them
    df = df.loc[:, ~df.columns.duplicated(keep=keep)]

    # Optional logging
    if log and dupes:
        print("Removed duplicate columns:", dupes)

    return df


In [9]:
# def prepend_empty_columns(df, col_list):
def prepend_empty_columns(df, col_list):
    """
    Prepend empty columns (from col_list) to the front of df.
    Returns a new DataFrame.
    """
    import pandas as pd

    # Create empty columns with same row count
    empty_df = pd.DataFrame(
        {col: [None] * len(df) for col in col_list}
    )

    # Prepend them
    return pd.concat([empty_df, df], axis=1)


In [10]:
# revised def fill_ib_media_cols(df):
def fill_ib_media_cols(df):
    df = df.copy()

    # --- 1. Clean and parse timestamp --------------------------
    def fix_ts(x):
        if pd.isna(x):
            return np.nan
        # Convert float → int safely
        try:
            x_int = int(float(x))
        except:
            return np.nan
        # Zero‑pad to 14 digits (YYYYMMDDHHMMSS)
        s = str(x_int).zfill(14)
        return pd.to_datetime(s, format="%Y%m%d%H%M%S", errors="coerce")

    df['timestamp'] = df['Test Date / Time'].apply(fix_ts)

    # --- 2. dtv ------------------------------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id ----------------------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 12 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4–5. cls, cmmnts --------------------------------------
    df['cls'] = np.nan
    df['cmmnts'] = np.nan

    return df


In [11]:
# Sort the rows by timestamp
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [12]:
# remove duplicates on the basis of timestamp
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


In [13]:
# veirfy if rows exixt in master_timestamps(df_master, df_new, ts_col="timestamp"):
def filter_new_rows_by_master_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Filters df_new so that only rows whose timestamps appear in df_master remain.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing valid timestamps.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows whose timestamps
        exist in df_master.
    """

    # Extract the set of valid timestamps from the master dataframe
    valid_timestamps = set(df_master[ts_col])

    # Filter df_new to keep only rows with timestamps in the master set
    df_filtered = df_new[df_new[ts_col].isin(valid_timestamps)].copy()

    return df_filtered


In [14]:
# def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp")
def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp"):
    """
    Returns only the rows in df_new whose timestamps do NOT exist in df_master.

    Parameters
    ----------
    df_master : pd.DataFrame
        The master dataframe containing timestamps already ingested.
    df_new : pd.DataFrame
        The new dataframe to be filtered.
    ts_col : str, optional
        The name of the timestamp column (default is 'timestamp').

    Returns
    -------
    pd.DataFrame
        A filtered version of df_new containing only rows with timestamps
        NOT present in df_master.
    """

    # Extract the set of timestamps already in the master
    existing_ts = set(df_master[ts_col])

    # Keep only rows whose timestamp is NOT in the master
    df_filtered = df_new[~df_new[ts_col].isin(existing_ts)].copy()

    return df_filtered


In [15]:
# def append_rows_with_master_schema(master_df, adder_df):
import pandas as pd
import numpy as np

def append_rows_with_master_schema(master_df, adder_df):
    """
    Appends rows from adder_df into master_df while enforcing the master_df schema.

    For each row in adder_df:
      - Columns that exist in adder_df are copied.
      - Columns missing from adder_df are filled with NaN.
      - All master_df columns are preserved in order.

    Parameters
    ----------
    master_df : pd.DataFrame
        The master dataframe with the full schema.
    adder_df : pd.DataFrame
        The dataframe containing rows to append (subset of master columns).

    Returns
    -------
    pd.DataFrame
        Updated master_df with new rows appended.
    """

    # Reindex adder_df to match master_df columns, filling missing columns with NaN
    adder_aligned = adder_df.reindex(columns=master_df.columns)

    # Append and return
    return pd.concat([master_df, adder_aligned], ignore_index=True)


In [16]:
# def filter_by_value(df, column, value):
def filter_by_value(df, column, value):
    """
    Returns a filtered DataFrame containing only rows where df[column] == value.

    Parameters
    ----------
    df : pd.DataFrame
        The dataframe to filter.
    column : str
        The column name to filter on.
    value : any
        The value that the column must match.

    Returns
    -------
    pd.DataFrame
        A filtered dataframe containing only matching rows.
    """
    return df[df[column] == value].copy()


# UPDATE **"df_mf_ib_tst"** with latest data

## Bringing in the main data file from Excel and produce clean **df_mf_ib_tst_nn** 

### load from XL

In [75]:
#this brings in the main data file from Excel
df_mf_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/mf_ib_tst.csv")
df_mf_ib_tst_nn = strip_numbers_from_columns(df_mf_ib_tst)
# verify 
df_mf_ib_tst_nn
print("imported **df_m_ib_tst_nn** ie numbers removed from col heads")

imported **df_m_ib_tst_nn** ie numbers removed from col heads


### Insure no numbers in col names & slice-abilty

In [76]:
# df_m_ib_tst_nn slicing
# verify print('df_m_ib_tst_nn imported \n Test Date / Time \n',df_mf_ib_tst_nn['Test Date / Time'])  # list form wo meta


### Insure that media_lst cols inserted in **df_m_ib_tst_nn**

In [77]:
# verify df_mf_ib_tst_nn[media_lst]

### Insure that media_lst cols are filled in **df_mf_ib_tst_nn**

In [78]:
# fill_ib_media_cols(df_mf_ib_tst_nn)
df_mf_ib_tst_nn = fill_ib_media_cols(df_mf_ib_tst_nn)
# verify df_mf_ib_tst_nn

### sort **df_mf_ib_tst_nn** On timestamp to get **df_mf_ib_tst_nn_s**

In [79]:
# def sort_by_timestamp(df_mf_ib_tst_nn):
df_mf_ib_tst_nn_s = sort_by_timestamp(df_mf_ib_tst_nn)
# vefify df_mf_ib_tst_nn_s

## Import new data from ip77 in the data folder of repo and produce clean rows ready to concatinate

### This segment imports the data from the Excel file probo/data/ib77 to dataframe with numbers in col names

In [22]:
# loads the new from the 770
df_ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# verify df_ib77_raw #print("df_ib77_raw w/o numbers OK")

✅ Loaded 091725_20251225081906.csv with utf-8
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251229081757.csv with utf-8
✅ Loaded 091725_20251223231059.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251221071026.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251221205127.csv with utf-8
✅ Loaded 091725_20251226083412.csv with utf-8
✅ Loaded 091725_20251227083952.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251228225544.csv with utf-8
✅ Loaded 091725_20251224080342.csv with utf-8
✅ Loaded 091725_20251222221501.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.c

### This segment strips off the col names of numbers and produces "df_m_ib77_nn" and demonstrates slicing

In [80]:
df_ib77_nn = strip_numbers_from_columns(df_ib77_raw)
#verify print(list(df_ib77_nn.columns))

In [81]:
# verify slicing df_ib77_nn[['ID','Test Date / Time','ECW/TBW']]  #,"SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

In [82]:
df_ib77_nn_s = df_ib77_nn.sort_values(by="Test Date / Time").reset_index(drop=True)

In [83]:
# Seek out and drop duplicates
df_ib77_nn_s = drop_duplicate_columns(df_ib77_nn_s)

Removed duplicate columns: ['Systolic', 'Diastolic', 'Pulse', 'Mean Artery Pressure', 'Pulse Pressure', 'Rate Pressure Product']


In [84]:
#df_ib77_nn_s[['ID','Test Date / Time','ECW/TBW']]

### This segment eliminates duplicates in **df_ib77_nn**

In [85]:
# Seek out and drop duplicates
df_ib77_nn = drop_duplicate_columns(df_ib77_nn)

Removed duplicate columns: ['Systolic', 'Diastolic', 'Pulse', 'Mean Artery Pressure', 'Pulse Pressure', 'Rate Pressure Product']


In [86]:
# Testing duplicates in the column titles
df_ib77_nn.columns[df_ib77_nn.columns.duplicated()]

print ("df_ib77_nn , No numbers, no duplicates OK")

df_ib77_nn , No numbers, no duplicates OK


### This segment adds **media_cols** and fills them from data in the results **"df_m_ib77_nn"**

In [87]:
df_m_ib77_nn = prepend_empty_columns(df_ib77_nn, media_lst)
# verify 
df_m_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251229081757.csv,utf-8
4,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
5,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
6,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
7,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
8,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
9,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8


### This segment eliminates duplicates in **df_m_ib77_nn**

In [88]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)
print ("df_m_ib77_nn , No numbers, no duplicates OK")

df_m_ib77_nn , No numbers, no duplicates OK


In [89]:
# verify df_m_ib77_nn[media_lst]

In [90]:
df_m_ib77_nn = drop_duplicate_columns(df_m_ib77_nn)

In [91]:
df_m_ib77_nn.columns[df_m_ib77_nn.columns.duplicated()]
print("No duplicates detected")

No duplicates detected


In [92]:
# verify that ['Test Date / Time'] exists
df_m_ib77_nn

,timestamp,dtv,ib_id,cls,cmmnts,Name,ID,Height,Date of Birth,Age,...,R/Ht,Xc/Ht,HGS of Left Arm 1st,HGS of Left Arm 2nd,HGS of Right Arm 1st,HGS of Right Arm 2nd,HGS/WT,Unnamed,source_file,encoding_used
0,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251225081906.csv,utf-8
1,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251213054921.csv,utf-8
2,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251214234814.csv,utf-8
3,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251229081757.csv,utf-8
4,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251223231059.csv,utf-8
5,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251217075000.csv,utf-8
6,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251218081604.csv,utf-8
7,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251221071026.csv,utf-8
8,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251216233809.csv,utf-8
9,None,None,None,None,None,91725,6ft 06.0in,1939.02.23.,F,86.0,...,0.0,0.0,-,-,-,-,-,NaN,091725_20251215080038.csv,utf-8


In [93]:
# fill_ib_media_cols(df_mf_ib77_nn)
df_mf_ib77_nn = fill_ib_media_cols(df_m_ib77_nn)
# verify df_mf_ib77_nn

### This segment eliminates duplicates based on **"test_time" in **"df_mf_ib77_nn"** note: "mf" means media added and filled]

In [94]:
df_mf_ib77_nn.columns[df_mf_ib77_nn.columns.duplicated()]
print("No duplicates **df_mf_ib77_nn** detected")

No duplicates **df_mf_ib77_nn** detected


In [96]:
# verify df_mf_ib77_nn[media_lst]

### THIS IS TEMP OUT >> This segment sorts based on **"timestamp"** in **"df_mf_ib77_nn"** with results **df_mf_ib77_nn_s**

In [97]:
#sort_by_timestamp(df_mf_ib77_nn)
df_mf_ib77_nn_s = sort_by_timestamp(df_mf_ib77_nn)
#verify 
df_mf_ib77_nn_s[["timestamp",'ib_id',"ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'VFA (Visceral Fat Area)']]

,timestamp,ib_id,ECW/TBW,BMR (Basal Metabolic Rate),SMM (Skeletal Muscle Mass),VFA (Visceral Fat Area)
0,2025-12-11 09:26:10,mrn,0.410,2158,99.4,167.0
1,2025-12-12 08:42:31,mrn,0.409,2186,101.2,162.7
2,2025-12-12 22:17:13,eve,0.415,2208,101.6,158.6
3,2025-12-13 05:49:21,mrn,0.410,2203,102.3,159.8
4,2025-12-14 09:03:36,mrn,0.407,2148,99.2,167.3
5,2025-12-14 23:48:14,eve,0.414,2160,99.2,163.1
6,2025-12-15 08:00:38,mrn,0.406,2155,99.9,166.8
7,2025-12-15 23:33:24,eve,0.413,2230,103.4,153.1
8,2025-12-16 06:48:08,mrn,0.406,2129,98.3,167.5
9,2025-12-16 23:38:09,eve,0.411,2141,98.3,166.4


## Eliminate all rows that have time stamps equal to a timestamp in the master data frame: **df_mf_ib_tst** yielding **df_mf_ib77_nn_sn** ....*"sn"* means *"sorted new"*

In [98]:
# Eliminate all rows that added to the master data frame

df_mf_ib77_nn_sn = keep_only_new_timestamps(
    df_mf_ib_tst_nn,
    df_mf_ib77_nn_s,
    ts_col="timestamp"
)
# verify df_mf_ib77_nn_sn

## Append Each Row And fill on a column by column And the missing columns with nan

In [99]:
df_mf_ib_tst = append_rows_with_master_schema(
    df_mf_ib_tst_nn_s,
    df_mf_ib77_nn_sn
)
# verify df_mf_ib_tst


In [100]:
df_mf_ib_tst = df_mf_ib_tst.sort_values(by="Test Date / Time").reset_index(drop=True)
# df_mf_ib_tst

In [101]:
# Eliminate all rows that added to the master data frame
#def keep_only_new_timestamps(df_master, df_new, ts_col="timestamp"):
df_mf_ib_tst_s = keep_only_new_timestamps(
    df_mf_ib_tst,
    df_mf_ib_tst_s,
    ts_col="timestamp"
)
# verify df_mf_ib_tst_s

# Write the value to both pickle and to the Excel file, the end result is all stored in the master:  
1. master file called **mf_ib_tst**
2. a cleaned master df called **df_mf_ib_tst_nn_s**
3. the add df source file valled **ib97**
4. updated with a cleaned df **df_mf_ib77_nn_sn**
5. add data path **"/home/bhuns/JL_2/data/ib97**
6. master path **"/home/bhuns/JL_2/data/ib_tst/mf_ib_tst.csv"**

# WORKING cell 
1. df_mf_ib_tst_nn_s
2. df_mf_ib77_nn_sn



In [40]:
list(df_mf_ib77_nn_sn.columns)

['timestamp',
 'dtv',
 'ib_id',
 'cls',
 'cmmnts',
 'Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'TBW (Total Body Water)',
 'ICW (Intracellular Water)',
 'ECW (Extracellular Water)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'LBM (Lean Body Mass)',
 'SMM (Skeletal Muscle Mass)',
 'BMI (Body Mass Index)',
 'PBF (Percent Body Fat)',
 'LBM of Right Arm',
 'LBM% of Right Arm',
 'LBM of Left Arm',
 'LBM% of Left Arm',
 'LBM of Trunk',
 'LBM% of Trunk',
 'LBM of Right Leg',
 'LBM% of Right Leg',
 'LBM of Left Leg',
 'LBM% of Left Leg',
 'Leg Lean Mass',
 'TBW of Right Arm',
 'TBW of Left Arm',
 'TBW of Trunk',
 'TBW of Right Leg',
 'TBW of Left Leg',
 'ICW of Right Arm',
 'ICW of Left Arm',
 'ICW of Trunk',
 'ICW of Right Leg',
 'ICW of Left Leg',
 'ECW of Right Arm',
 'ECW of Left Arm',
 'ECW of Trunk',
 'ECW of Right Leg',
 'ECW of Left Leg

In [69]:
df_mf_ib_tst_nn_s[["timestamp",'ib_id',"ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'kHz-RL Reactance','VFA (Visceral Fat Area)'
]]

,timestamp,ib_id,ECW/TBW,BMR (Basal Metabolic Rate),SMM (Skeletal Muscle Mass),kHz-RL Reactance,VFA (Visceral Fat Area)
0,2025-09-22 22:58:26,eve,0.413,2151,98.3,NaN,165.9
1,2025-12-11 09:26:10,mrn,0.410,2158,99.4,6.0,167.0
4,2025-12-12 08:42:31,mrn,0.409,2186,101.2,6.3,162.7
7,2025-12-12 22:17:13,eve,0.415,2208,101.6,4.6,158.6
10,2025-12-13 05:49:21,mrn,0.410,2203,102.3,5.8,159.8
13,2025-12-14 09:03:36,mrn,0.407,2148,99.2,6.7,167.3
16,2025-12-14 23:48:14,eve,0.414,2160,99.2,4.5,163.1
19,2025-12-15 08:00:38,mrn,0.406,2155,99.9,7.5,166.8
22,2025-12-15 23:33:24,eve,0.413,2230,103.4,5.1,153.1
25,2025-12-16 06:48:08,mrn,0.406,2129,98.3,7.2,167.5


In [47]:
# Seek out and drop duplicates
df_mf_ib_tst_nn_s = drop_duplicate_columns(df_mf_ib_tst_nn_s)

In [44]:
df_mf_ib_tst_nn_s[["timestamp",'ib_id',"ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'kHz-RL Reactance','VFA (Visceral Fat Area)']]
]]

,timestamp,ib_id,ECW/TBW,BMR (Basal Metabolic Rate),SMM (Skeletal Muscle Mass),kHz-RL Reactance,VFA (Visceral Fat Area)
0,2025-09-22 22:58:26,eve,0.413,2151,98.3,NaN,165.9
1,2025-12-11 09:26:10,mrn,0.410,2158,99.4,6.0,167.0
2,2025-12-11 09:26:10,mrn,0.410,2158,99.4,6.0,167.0
3,2025-12-11 09:26:10,mrn,0.410,2158,99.4,6.0,167.0
4,2025-12-12 08:42:31,mrn,0.409,2186,101.2,6.3,162.7
5,2025-12-12 08:42:31,mrn,0.409,2186,101.2,6.3,162.7
6,2025-12-12 08:42:31,mrn,0.409,2186,101.2,6.3,162.7
7,2025-12-12 22:17:13,eve,0.415,2208,101.6,4.6,158.6
8,2025-12-12 22:17:13,eve,0.415,2208,101.6,4.6,158.6
9,2025-12-12 22:17:13,eve,0.415,2208,101.6,4.6,158.6


In [46]:
df_mf_ib_tst_nn_s = drop_duplicate_columns(df_mf_ib_tst_nn_s)

In [102]:
# detect duplicate rows and keep the first one
df_mf_ib_tst_nn_s = df_mf_ib_tst_nn_s.drop_duplicates(
    subset=['timestamp'],
    keep='first'
)
#verify df_mf_ib_tst_nn_s[["timestamp",'ib_id',"ECW/TBW", 'BMR (Basal Metabolic Rate)',"SMM (Skeletal Muscle Mass)",'kHz-RL Reactance','VFA (Visceral Fat Area)']]

In [103]:
df_mf_ib_tst_nn_s_mrn = filter_by_value(
    df_mf_ib_tst_nn_s,
    "ib_id",
    "mrn"
)


# Converting data for use in ploting 
1. need to convert by storing to pickle
2. need to convert by creating name file

In [ ]:
# Load "ib970mrn_grp1" from pickle
import pickle
with open("ib970mrn_grp1.pkl", "rb") as f: 'BMR (Basal Metabolic Rate)', 'BMR (Basal Metabolic Rate)',
    ib970mrn_grp1 = pickle.load(f)
print("ib970mrn_grp1 has been loaded from pickle")

In [ ]:
# Load "ib970mrn_grp1_nms" from pickle
import pickle
with open("ib970mrn_grp1_nms.pkl", "rb") as f:
    ib970mrn_grp1_nms = pickle.load(f)
print("ib970mrn_grp1_nms has been loaded from pickle")